In [1]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

courses_collection.delete_many({})

operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),
    
    UpdateOne(
        {'course': 'CS 102'},
        {'$set': {'enrollments': 32}}
    ),

    DeleteOne({'enrollments': {'$lt': 18}})
]

result = courses_collection.bulk_write(operations)
print("Bulk write completed.")

print("\n=== FINAL DATA ===")
for course in courses_collection.find():
    print(course)

Bulk write completed.

=== FINAL DATA ===
{'_id': ObjectId('690ea83bec6c939e1cdcb01b'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690ea83bec6c939e1cdcb01c'), 'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science'}
{'_id': ObjectId('690ea83bec6c939e1cdcb01d'), 'course': 'History 201', 'enrollments': 20, 'department': 'History'}


In [2]:
#2A
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)


{'_id': ObjectId('690ea83bec6c939e1cdcb01b'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690ea83bec6c939e1cdcb01c'), 'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science'}


In [3]:
#2B
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

{'_id': ObjectId('690ea83bec6c939e1cdcb01b'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690ea83bec6c939e1cdcb01c'), 'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science'}


In [4]:
#3A
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'Computer Science', 'average_enrollment': 32.0}
{'_id': 'History', 'average_enrollment': 20.0}
{'_id': 'Mathematics', 'average_enrollment': 30.0}


In [6]:
#3B
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'Computer Science', 'max_enrollment': 32}
{'_id': 'History', 'max_enrollment': 20}
{'_id': 'Mathematics', 'max_enrollment': 30}


In [8]:
#4A
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690ea83bec6c939e1cdcb01b'), 'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('690ea83bec6c939e1cdcb01c'), 'enrollments': 32, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('690ea83bec6c939e1cdcb01d'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}


In [12]:
#4B
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690ea83bec6c939e1cdcb01b'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('690ea83bec6c939e1cdcb01c'), 'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('690ea83bec6c939e1cdcb01d'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}


In [18]:
#3
print("\n=== Lookup Courses dengan Students ===")

pipeline = [
    {'$lookup': {
        'from': 'students',
        'localField': 'course_id',
        'foreignField': 'course_id',
        'as': 'daftar_mahasiswa'
    }}
]

for course in courses_collection.aggregate(pipeline):
    print(course)


=== Lookup Courses dengan Students ===
{'_id': ObjectId('690ea83bec6c939e1cdcb01b'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'daftar_mahasiswa': [{'_id': ObjectId('690c06c24a642dfbe6e6b03b'), 'student_id': 1001, 'name': 'Andi'}, {'_id': ObjectId('690c06c24a642dfbe6e6b03c'), 'student_id': 1002, 'name': 'Budi'}, {'_id': ObjectId('690c06c24a642dfbe6e6b03d'), 'student_id': 1003, 'name': 'Citra'}, {'_id': ObjectId('690c06c24a642dfbe6e6b03e'), 'student_id': 1004, 'name': 'Dewi'}, {'_id': ObjectId('690c06c24a642dfbe6e6b03f'), 'student_id': 1005, 'name': 'Eka'}, {'_id': ObjectId('690c06c24a642dfbe6e6b040'), 'student_id': 1006, 'name': 'Farhan'}, {'_id': ObjectId('690c06c24a642dfbe6e6b041'), 'student_id': 1007, 'name': 'Gita'}, {'_id': ObjectId('690c06c24a642dfbe6e6b042'), 'student_id': 1008, 'name': 'Hadi'}, {'_id': ObjectId('690c06c24a642dfbe6e6b043'), 'student_id': 1009, 'name': 'Indah'}, {'_id': ObjectId('690c06c24a642dfbe6e6b044'), 'student_id': 1010, 'name':

In [26]:
students = db['students']

students.delete_many({})

students_data = [
    {'student_id': 1, 'name': "Dina", 'major': "Mathematics", 'course_id': 1},
    {'student_id': 2, 'name': "Fajar", 'major': "Mathematics", 'course_id': 1},
    {'student_id': 3, 'name': "Indra", 'major': "Computer Science", 'course_id': 2},
    {'student_id': 4, 'name': "Samsul", 'major': "Computer Science", 'course_id': 2},
    {'student_id': 5, 'name': "joko", 'major': "Chemical Engineering", 'course_id': 10},
    {'student_id': 6, 'name': "Eldyss", 'major': "History", 'course_id': 3},
    {'student_id': 7, 'name': "Buana", 'major': "Information Technology", 'course_id': 11},
    {'student_id': 8, 'name': "Kaesang", 'major': "Civil Engineering", 'course_id': 12},
    {'student_id': 9, 'name': "Gibran", 'major': "Philosophy", 'course_id': 13},
    {'student_id': 10, 'name': "Uskho", 'major': "Mathematics Science", 'course_id': 14},
    {'student_id': 11, 'name': "Mahardika", 'major': "Physics", 'course_id': 4},
    {'student_id': 12, 'name': "Farrel", 'major': "Software Engineering", 'course_id': 15},
]

students.insert_many(students_data)

print("=== ᴅᴀᴛᴀ ꜱᴛᴜᴅᴇɴᴛꜱ ꜱᴀᴀᴛ ɪɴɪ ===")
for student in students.find({}, {'_id': 0}):
    print(student)


=== ᴅᴀᴛᴀ ꜱᴛᴜᴅᴇɴᴛꜱ ꜱᴀᴀᴛ ɪɴɪ ===
{'student_id': 1, 'name': 'Dina', 'major': 'Mathematics', 'course_id': 1}
{'student_id': 2, 'name': 'Fajar', 'major': 'Mathematics', 'course_id': 1}
{'student_id': 3, 'name': 'Indra', 'major': 'Computer Science', 'course_id': 2}
{'student_id': 4, 'name': 'Samsul', 'major': 'Computer Science', 'course_id': 2}
{'student_id': 5, 'name': 'joko', 'major': 'Chemical Engineering', 'course_id': 10}
{'student_id': 6, 'name': 'Eldyss', 'major': 'History', 'course_id': 3}
{'student_id': 7, 'name': 'Buana', 'major': 'Information Technology', 'course_id': 11}
{'student_id': 8, 'name': 'Kaesang', 'major': 'Civil Engineering', 'course_id': 12}
{'student_id': 9, 'name': 'Gibran', 'major': 'Philosophy', 'course_id': 13}
{'student_id': 10, 'name': 'Uskho', 'major': 'Mathematics Science', 'course_id': 14}
{'student_id': 11, 'name': 'Mahardika', 'major': 'Physics', 'course_id': 4}
{'student_id': 12, 'name': 'Farrel', 'major': 'Software Engineering', 'course_id': 15}


In [29]:
#TUGAS 1
print("\n=== Total Course Setiap Department ===")

pipeline_total_course = [
    {'$group': {
        '_id': '$department',
        'total_courses': {'$sum': 1}
    }},
    {'$project': {
        '_id': 0,
        'department': '$_id',
        'total_courses': 1
    }}
]

for result in courses_collection.aggregate(pipeline_total_course):
    print(result)


=== Total Course Setiap Department ===
{'total_courses': 9, 'department': 'Computer Science'}
{'total_courses': 2, 'department': 'History'}
{'total_courses': 6, 'department': 'Mathematics'}
{'total_courses': 1, 'department': 'Physics'}


In [30]:
#TUGAS 2
print("\n=== Course COMPUTER SCIENCE dengan Enrollments > 25 ===")

pipeline_filter_cs = [
    {'$match': {
        'department': 'Computer Science',
        'enrollments': {'$gt': 25}
    }},
    {'$group': {
        '_id': '$department',
        'jumlah_course_lolos': {'$sum': 1}
    }}
]

for result in courses_collection.aggregate(pipeline_filter_cs):
    print(result)



=== Course COMPUTER SCIENCE dengan Enrollments > 25 ===
{'_id': 'Computer Science', 'jumlah_course_lolos': 7}


In [ ]:
#TUGAS 3
import pandas as pd

lookup_stage = [
    {
        '$lookup': {
            'from': 'students',
            'localField': 'course_id',
            'foreignField': 'course_id',
            'as': 'students_info'
        } } ]

project_stage = [
    {
        '$project': {
            '_id': 0,
            'students_info._id': 0
        }
    }
]

pipeline = lookup_stage + project_stage

result = list(courses_collection.aggregate(pipeline))
df = pd.DataFrame(result)

# --- Hitung jumlah mahasiswa di setiap course ---
df['student_count'] = df['students_info'].apply(lambda x: len(x))

# --- Hapus kolom students_info karena berisi array ---
df_clean = df.drop(columns=['students_info'])

print(df_clean)
